In [1]:
# -*- coding: utf-8 -*-
# %%
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %% [markdown]
# ### Install packages and import
# %%
# #################################### PLEASE INSTALL LATEST CHROME WEBDRIVER #####################################
# Uncomment to run as required
# #     --install-option="--chromedriver-version= *.**" \
#   --install-option="--chromedriver-checksums=4fecc99b066cb1a346035bf022607104,058cd8b7b4b9688507701b5e648fd821"
# %%
# ##### COPY THE LINES IN THIS COMMENT TO THE TOP OF NEW SCRIPTS #####
# # Function to import this package to other files
# import os
# import sys
# from pathlib import Path

# code_dir = None
# code_dir_name = 'Code'
# unwanted_subdir_name = 'Analysis'

# for _ in range(5):

#     parent_path = str(Path.cwd().parents[_]).split('/')[-1]

#     if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

#         code_dir = str(Path.cwd().parents[_])

#         if code_dir is not None:
#             break

# main_dir = str(Path(code_dir).parents[0])
# scraped_data = f'{code_dir}/scraped_data'
# sys.path.append(code_dir)

# from setup_module.imports import *
# from setup_module.params import *
# from setup_module.scraping import *
# from setup_module.classification import *
# from setup_module.vectorizers_classifiers import *

# warnings.filterwarnings("ignore", category=DeprecationWarning)
# %matplotlib notebook
# %matplotlib inline

# %%
import os
import sys
from pathlib import Path

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

main_dir = str(Path(code_dir).parents[0])
scraped_data = f'{code_dir}/scraped_data'
sys.path.append(code_dir)

from setup_module.imports import *
from setup_module.params import *
from setup_module.scraping import *
from setup_module.classification import *
from setup_module.vectorizers_classifiers import *

warnings.filterwarnings("ignore")

%matplotlib notebook
%matplotlib widget
%matplotlib inline

File /Users/nyxinsane/Library/CloudStorage/OneDrive-UvA/Automating Equity/Study 1/Study1_Code/Sectors + Age and Gender Composition of Industires and Jobs/ not found.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enabled.
Printing keywords not enable

In [ ]:
if save_enabled is True:
    try:
        table_df = pd.read_csv(table_save_path + csv_file_name, index_col=[0], header=[0,1,2], skipinitialspace=True)
    except Exception:
        table_df = pd.read_excel(table_save_path + excel_file_name, index_col=[0], header=[0,1,2])
    except Exception:
        table_df = pd.read_pickle(table_save_path + pickle_file_name, index_col=[0], header=[0,1,2])

# table_df.loc[slice(None), (slice(None), slice(None), 'Confusion Matrix')]

In [ ]:
df = pd.read_csv(f'{df_dir}df_processed_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}')

for col in tqdm.tqdm(analysis_columns):

    print('-' * 20)
    print('\n')
    print(f'============================ PROCESSING {col} ============================')
    print('\n')
    print('-' * 20)

    if str(col) == 'Warmth':
        vectorizer_name = final_warmth_vectorizer
        classifier_name = final_warmth_classifier
    elif str(col) == 'Competence':
        vectorizer_name = final_competence_vectorizer
        classifier_name = final_competence_classifier

    vectorizer_loc = (f'FINAL Vectorizer {col} - {vectorizer_name} + {classifier_name}.{file_save_format}')
    if select_best_enabled is True:
        selector = 'SelectKBest'
        selector_loc = (
            f'FINAL Selector {col} - {vectorizer_name} + {classifier_name}.{file_save_format}'
        )
    model_loc = (f'Model {col} - {vectorizer_name} + {classifier_name}.{file_save_format}')

    # Plot Confusion Matrix
    cm = table_df.loc[(classifier_name), (col, vectorizer_name, 'Confusion Matrix')]
    if isinstance(cm, str):
        cm = cm.replace('[','').replace(']','')
        cm = np.fromstring(cm,dtype=int,sep=' ').reshape((-1, 2))
    heatmap = plot_confusion_matrix_percentage(col, cm, classifier_name, vectorizer_name)
    # plt.tight_layout()
    plt.show()
    if save_enabled is True:
        heatmap.figure.savefig(
            f'{plot_save_path}Confusion Matrix {str(col)} - {classifier_name} - {vectorizer_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000,
            )
    plt.clf()
    plt.cla()
    plt.close()

    with open(models_save_path + model_loc, 'rb') as f:
        model = joblib.load(f)
    with open(models_save_path + vectorizer_loc, 'rb') as f:
        vectorizer = joblib.load(f)
    if select_best_enabled is True:
        with open(models_save_path + selector_loc, 'rb') as f:
            selector = joblib.load(f)

    # BOW transform
    X_test = df[str(text_col)].astype("str").values
    features = vectorizer.transform(X_test)
    if select_best_enabled is True:
        features = selector.transform(features)
    y_pred = model.predict(features)
    df[f'{str(col)}'] = y_pred
    if hasattr(model, 'predict_proba'):
        y_test_prob_pred = model.predict_proba(features)
        probability_of_1 = y_test_prob_pred[:, 1]
        df[f'{str(col)}_Probability'] = probability_of_1

In [ ]:
df = categorize_df_gender_age(df)
df.dropna(subset=['Warmth', 'Warmth_Probability', 'Competence', 'Competence_Probability'], inplace=True)

In [ ]:
df_gender_age_info(df, print_info=True)
df_warm_comp_info(df, print_info=True)


In [ ]:
df_gender_age_info(df, print_info=True)

In [ ]:
df.describe()

In [ ]:
df.to_pickle(
    f'{df_dir}df_classified_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')

df.to_csv(
    f'{df_dir}df_classified_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}',
    index=False,
)